In [ ]:
# Cell 1: Overview Table# Summary table showing key metrics for all parameter combinationsimport jsonimport pandas as pdimport numpy as np# Load datawith open('parameter_sweep_data.json', 'r') as f:    data = json.load(f)# Create summary tablesummary_data = []for run in data['data']:    # Calculate mean temperature from T_final (at layer centers)    # T_final is at interfaces, need to get layer centers    T_final_interfaces = np.array(run['T_final'])    T_centers = (T_final_interfaces[:-1] + T_final_interfaces[1:]) / 2.0        summary_data.append({        'alpha': run['alpha'],        'timestep': run['timestep'],        'steps_to_converge': run['steps_to_converge'],        'converged': run['converged'],        'n_convective_final': run['n_convective_final'],        'max_grad_diff_final': run['max_grad_diff_final'],        'mean_T_final': np.mean(T_centers),        'min_T_final': np.min(T_centers),        'max_T_final': np.max(T_centers)    })df_summary = pd.DataFrame(summary_data)# Sort by alpha, then timestep for easy readingdf_summary = df_summary.sort_values(['alpha', 'timestep'])# Display formatted tableprint("=" * 100)print("Parameter Effects Summary Table")print("=" * 100)print(f"\nTotal parameter combinations: {len(df_summary)}")print(f"\n{df_summary.to_string(index=False)}\n")# Print summary statisticsprint("\n" + "=" * 100)print("Summary Statistics by Parameter")print("=" * 100)# By alphaprint("\nBy Alpha (across all timesteps):")alpha_stats = df_summary.groupby('alpha').agg({    'steps_to_converge': ['mean', 'min', 'max'],    'n_convective_final': ['mean', 'min', 'max'],    'max_grad_diff_final': ['mean', 'min', 'max'],    'mean_T_final': ['mean', 'std']}).round(2)print(alpha_stats)# By timestepprint("\nBy Timestep (across all alpha values):")timestep_stats = df_summary.groupby('timestep').agg({    'steps_to_converge': ['mean', 'min', 'max'],    'n_convective_final': ['mean', 'min', 'max'],    'max_grad_diff_final': ['mean', 'min', 'max'],    'mean_T_final': ['mean', 'std']}).round(2)print(timestep_stats)

In [ ]:
# Cell 2: Temperature Profile Comparison (Fixed timestep, varying alpha)# Compare layer-by-layer temperature profiles for different alpha values at fixed timestepimport jsonimport numpy as npimport matplotlib.pyplot as plt# Load datawith open('parameter_sweep_data.json', 'r') as f:    data = json.load(f)# Get unique timestep valuestimesteps = sorted(set(run['timestep'] for run in data['data']))alphas = sorted(set(run['alpha'] for run in data['data']))# Create subplots: one per timestepn_timesteps = len(timesteps)fig, axes = plt.subplots(1, n_timesteps, figsize=(5*n_timesteps, 8), sharey=True)if n_timesteps == 1:    axes = [axes]# Colors for different alpha valuescolors = plt.cm.viridis(np.linspace(0, 1, len(alphas)))for timestep_idx, timestep in enumerate(timesteps):    ax = axes[timestep_idx]        # Filter runs for this timestep    runs_at_timestep = [run for run in data['data'] if abs(run['timestep'] - timestep) < 1e-6]        # Plot one line per alpha value    for alpha_idx, alpha in enumerate(alphas):        runs_at_alpha = [run for run in runs_at_timestep if abs(run['alpha'] - alpha) < 1e-6]        if len(runs_at_alpha) == 0:            continue                run = runs_at_alpha[0]                # Get temperature at layer centers and altitude        T_final_interfaces = np.array(run['T_final'])        z_interfaces_km = np.array(run['z_interfaces_km'])                # Calculate layer centers (average of adjacent interfaces)        T_centers = (T_final_interfaces[:-1] + T_final_interfaces[1:]) / 2.0        z_centers_km = (z_interfaces_km[:-1] + z_interfaces_km[1:]) / 2.0                ax.plot(T_centers, z_centers_km, 'o-', color=colors[alpha_idx],                label=f'α={alpha:.2f}', linewidth=2, markersize=4)        ax.set_xlabel('Temperature (K)', fontsize=12, fontweight='bold')    ax.set_ylabel('Altitude (km)', fontsize=12, fontweight='bold')    ax.set_title(f'Timestep = {timestep:.0f} s', fontsize=13, fontweight='bold')    ax.grid(True, alpha=0.3)    ax.legend(fontsize=9, loc='best')  #  ax.invert_yaxis()  # Invert Y-axis so altitude increases upward    ax.set_xlim(0,1800)plt.tight_layout()plt.suptitle('Temperature Profiles: Varying Alpha (Fixed Timestep)',              fontsize=14, fontweight='bold', y=1.02)plt.show()

In [ ]:
# Cell 3: Temperature Profile Comparison (Fixed alpha, varying timestep)# Compare layer-by-layer temperature profiles for different timesteps at fixed alphaimport jsonimport numpy as npimport matplotlib.pyplot as plt# Load datawith open('parameter_sweep_data.json', 'r') as f:    data = json.load(f)# Get unique timestep and alpha valuestimesteps = sorted(set(run['timestep'] for run in data['data']))alphas = sorted(set(run['alpha'] for run in data['data']))# Create subplots: one per alphan_alphas = len(alphas)fig, axes = plt.subplots(1, n_alphas, figsize=(5*n_alphas, 8), sharey=True)if n_alphas == 1:    axes = [axes]# Colors for different timestep valuescolors = plt.cm.plasma(np.linspace(0, 1, len(timesteps)))for alpha_idx, alpha in enumerate(alphas):    ax = axes[alpha_idx]        # Filter runs for this alpha    runs_at_alpha = [run for run in data['data'] if abs(run['alpha'] - alpha) < 1e-6]        # Plot one line per timestep value    for timestep_idx, timestep in enumerate(timesteps):        runs_at_timestep = [run for run in runs_at_alpha if abs(run['timestep'] - timestep) < 1e-6]        if len(runs_at_timestep) == 0:            continue                run = runs_at_timestep[0]                # Get temperature at layer centers and altitude        T_final_interfaces = np.array(run['T_final'])        z_interfaces_km = np.array(run['z_interfaces_km'])                # Calculate layer centers (average of adjacent interfaces)        T_centers = (T_final_interfaces[:-1] + T_final_interfaces[1:]) / 2.0        z_centers_km = (z_interfaces_km[:-1] + z_interfaces_km[1:]) / 2.0                ax.plot(T_centers, z_centers_km, 'o-', color=colors[timestep_idx],                label=f'dt={timestep:.0f}s', linewidth=2, markersize=4)        ax.set_xlabel('Temperature (K)', fontsize=12, fontweight='bold')    ax.set_ylabel('Altitude (km)', fontsize=12, fontweight='bold')    ax.set_title(f'Alpha = {alpha:.2f}', fontsize=13, fontweight='bold')    ax.grid(True, alpha=0.3)    ax.legend(fontsize=9, loc='best')    ax.set_xlim(0,1800)  #  ax.invert_yaxis()  # Invert Y-axis so altitude increases upwardplt.tight_layout()plt.suptitle('Temperature Profiles: Varying Timestep (Fixed Alpha)',              fontsize=14, fontweight='bold', y=1.02)plt.show()

In [ ]:
# Cell 4: Convective Layer Position Comparison# Heatmap showing which layers are convective/radiative across parametersimport jsonimport numpy as npimport matplotlib.pyplot as pltfrom matplotlib.colors import ListedColormapfrom matplotlib.patches import Rectangle, Patch# Load datawith open('parameter_sweep_data.json', 'r') as f:    data = json.load(f)# Get unique timestep and alpha valuestimesteps = sorted(set(run['timestep'] for run in data['data']))alphas = sorted(set(run['alpha'] for run in data['data']))# Get number of layers (assume all runs have same n_layers)n_layers = data['data'][0]['n_layers']def classify_layer_type_custom(N, N_ad):    """    Classify layers based on custom criterion:    - Convective: N > 1.5*N_ad    - Adiabatic: N_ad <= N <= 1.5*N_ad    - Radiative: N < N_ad    """    classifications = np.empty(len(N), dtype=object)        # Convective: N > 1.5*N_ad    convective_mask = N > 1.5 * N_ad    classifications[convective_mask] = 'convective'        # Adiabatic: N_ad <= N <= 1.5*N_ad    adiabatic_mask = (N >= N_ad) & (N <= 1.5 * N_ad)    classifications[adiabatic_mask] = 'adiabatic'        # Radiative: N < N_ad    radiative_mask = N < N_ad    classifications[radiative_mask] = 'radiative'        return classifications# Create two sets of plots# Plot 1: Fixed timestep, varying alphafig1, axes1 = plt.subplots(1, len(timesteps), figsize=(5*len(timesteps), 10), sharey=True)if len(timesteps) == 1:    axes1 = [axes1]# Custom colormap: convective=green, radiative=blue, adiabatic=yellowcolors_map = ['blue', 'yellow', 'green']  # radiative, adiabatic, convectivecmap = ListedColormap(colors_map)# Create legend elements (used for both figures)legend_elements = [    Patch(facecolor='green', label='Convective'),    Patch(facecolor='yellow', label='Adiabatic (N_ad ≤ N ≤ 1.5*N_ad)'),    Patch(facecolor='blue', label='Radiative'),    Patch(facecolor='white', edgecolor='red', hatch='///', linewidth=1.5, label='T > 3000K'),    Patch(facecolor='white', edgecolor='lightblue', hatch='///', linewidth=1.5, label='T < 100K')]for timestep_idx, timestep in enumerate(timesteps):    ax = axes1[timestep_idx]        # Filter runs for this timestep    runs_at_timestep = [run for run in data['data'] if abs(run['timestep'] - timestep) < 1e-6]        # Create grid: layers (rows) vs alpha (columns)    layer_type_grid = np.zeros((n_layers, len(alphas)), dtype=int)    high_T_mask = np.zeros((n_layers, len(alphas)), dtype=bool)  # Mask for T > 3000K    low_T_mask = np.zeros((n_layers, len(alphas)), dtype=bool)   # Mask for T < 100K        for alpha_idx, alpha in enumerate(alphas):        runs_at_alpha = [run for run in runs_at_timestep if abs(run['alpha'] - alpha) < 1e-6]        if len(runs_at_alpha) == 0:            continue                run = runs_at_alpha[0]                # Recalculate layer types using custom criterion (N_ad <= N <= 1.5*N_ad for adiabatic)        N_final = np.array(run['N_final'])        N_ad = run['N_ad']        layer_types = classify_layer_type_custom(N_final, N_ad)                # Get temperature at layer centers (average of adjacent interfaces)        T_final_interfaces = np.array(run['T_final'])        T_centers = (T_final_interfaces[:-1] + T_final_interfaces[1:]) / 2.0                # Map layer types to integers: radiative=0, adiabatic=1, convective=2        for layer_idx, layer_type in enumerate(layer_types):            if layer_type == 'radiative':                layer_type_grid[layer_idx, alpha_idx] = 0            elif layer_type == 'adiabatic':                layer_type_grid[layer_idx, alpha_idx] = 1            elif layer_type == 'convective':                layer_type_grid[layer_idx, alpha_idx] = 2                        # Check if temperature > 3000K            if T_centers[layer_idx] > 3000.0:                high_T_mask[layer_idx, alpha_idx] = True                        # Check if temperature < 100K            if T_centers[layer_idx] < 100.0:                low_T_mask[layer_idx, alpha_idx] = True        # Create heatmap    im = ax.imshow(layer_type_grid, aspect='auto', cmap=cmap, vmin=0, vmax=2,                    interpolation='nearest', origin='lower', extent=[-0.5, len(alphas)-0.5, -0.5, n_layers-0.5])        # Overlay hatching for temperature conditions    cell_width = 1.0    cell_height = 1.0    for layer_idx in range(n_layers):        for alpha_idx in range(len(alphas)):            # Red hatching for high temperature (T > 3000K)            if high_T_mask[layer_idx, alpha_idx]:                rect = Rectangle((alpha_idx - 0.5, layer_idx - 0.5), cell_width, cell_height,                               fill=False, hatch='///', edgecolor='red', linewidth=1.5, alpha=0.8)                ax.add_patch(rect)                        # Light blue hatching for low temperature (T < 100K)            if low_T_mask[layer_idx, alpha_idx]:                rect = Rectangle((alpha_idx - 0.5, layer_idx - 0.5), cell_width, cell_height,                               fill=False, hatch='///', edgecolor='lightblue', linewidth=1.5, alpha=0.8)                ax.add_patch(rect)        # Add vertical lines between alpha values (halfway between ticks)    for i in range(len(alphas) - 1):        ax.axvline(x=i + 0.5, color='white', linestyle='--', linewidth=1, alpha=1)        # Set ticks and labels    ax.set_xticks(range(len(alphas)))    ax.set_xticklabels([f'{alpha:.2f}' for alpha in alphas])    ax.set_yticks(range(0, n_layers, max(1, n_layers//10)))    ax.set_ylabel('Layer Index', fontsize=10)    ax.set_xlabel('Alpha', fontsize=10)    ax.set_title(f'Timestep = {timestep:.0f} s', fontsize=12, fontweight='bold')fig1.tight_layout()# Add legend to the first figurefig1.legend(handles=legend_elements, loc='right', bbox_to_anchor=(1.12, 0.5),            ncol=1, fontsize=10, frameon=True)plt.show()

In [ ]:
# Plot 2: Fixed alpha, varying timestep# Load data and set up if neededimport jsonimport numpy as npimport matplotlib.pyplot as pltfrom matplotlib.colors import ListedColormapfrom matplotlib.patches import Rectangle, Patchwith open('parameter_sweep_data.json', 'r') as f:    data = json.load(f)timesteps = sorted(set(run['timestep'] for run in data['data']))alphas = sorted(set(run['alpha'] for run in data['data']))n_layers = data['data'][0]['n_layers']def classify_layer_type_custom(N, N_ad):    classifications = np.empty(len(N), dtype=object)    convective_mask = N > 1.5 * N_ad    classifications[convective_mask] = 'convective'    adiabatic_mask = (N >= N_ad) & (N <= 1.5 * N_ad)    classifications[adiabatic_mask] = 'adiabatic'    radiative_mask = N < N_ad    classifications[radiative_mask] = 'radiative'    return classificationscolors_map = ['blue', 'yellow', 'green']cmap = ListedColormap(colors_map)legend_elements = [    Patch(facecolor='green', label='Convective'),    Patch(facecolor='yellow', label='Adiabatic (N_ad ≤ N ≤ 1.5*N_ad)'),    Patch(facecolor='blue', label='Radiative'),    Patch(facecolor='white', edgecolor='red', hatch='///', linewidth=1.5, label='T > 3000K'),    Patch(facecolor='white', edgecolor='lightblue', hatch='///', linewidth=1.5, label='T < 100K')]fig2, axes2 = plt.subplots(1, len(alphas), figsize=(5*len(alphas), 10), sharey=True)if len(alphas) == 1:    axes2 = [axes2]for alpha_idx, alpha in enumerate(alphas):    ax = axes2[alpha_idx]        # Filter runs for this alpha    runs_at_alpha = [run for run in data['data'] if abs(run['alpha'] - alpha) < 1e-6]        # Create grid: layers (rows) vs timestep (columns)    layer_type_grid = np.zeros((n_layers, len(timesteps)), dtype=int)    high_T_mask = np.zeros((n_layers, len(timesteps)), dtype=bool)    low_T_mask = np.zeros((n_layers, len(timesteps)), dtype=bool)        for timestep_idx, timestep in enumerate(timesteps):        runs_at_timestep = [run for run in runs_at_alpha if abs(run['timestep'] - timestep) < 1e-6]        if len(runs_at_timestep) == 0:            continue                run = runs_at_timestep[0]                # Recalculate layer types using custom criterion        N_final = np.array(run['N_final'])        N_ad = run['N_ad']        layer_types = classify_layer_type_custom(N_final, N_ad)                # Get temperature at layer centers        T_final_interfaces = np.array(run['T_final'])        T_centers = (T_final_interfaces[:-1] + T_final_interfaces[1:]) / 2.0                # Map layer types to integers        for layer_idx, layer_type in enumerate(layer_types):            if layer_type == 'radiative':                layer_type_grid[layer_idx, timestep_idx] = 0            elif layer_type == 'adiabatic':                layer_type_grid[layer_idx, timestep_idx] = 1            elif layer_type == 'convective':                layer_type_grid[layer_idx, timestep_idx] = 2                        if T_centers[layer_idx] > 3000.0:                high_T_mask[layer_idx, timestep_idx] = True                        if T_centers[layer_idx] < 100.0:                low_T_mask[layer_idx, timestep_idx] = True        # Create heatmap    im = ax.imshow(layer_type_grid, aspect='auto', cmap=cmap, vmin=0, vmax=2,                    interpolation='nearest', origin='lower', extent=[-0.5, len(timesteps)-0.5, -0.5, n_layers-0.5])        # Overlay hatching for temperature conditions    cell_width = 1.0    cell_height = 1.0    for layer_idx in range(n_layers):        for timestep_idx in range(len(timesteps)):            if high_T_mask[layer_idx, timestep_idx]:                rect = Rectangle((timestep_idx - 0.5, layer_idx - 0.5), cell_width, cell_height,                               fill=False, hatch='///', edgecolor='red', linewidth=1.5, alpha=0.8)                ax.add_patch(rect)                        if low_T_mask[layer_idx, timestep_idx]:                rect = Rectangle((timestep_idx - 0.5, layer_idx - 0.5), cell_width, cell_height,                               fill=False, hatch='///', edgecolor='lightblue', linewidth=1.5, alpha=0.8)                ax.add_patch(rect)        # Add vertical lines between timestep values    for i in range(len(timesteps) - 1):        ax.axvline(x=i + 0.5, color='white', linestyle='--', linewidth=1, alpha=1)        # Set ticks and labels    ax.set_xticks(range(len(timesteps)))    ax.set_xticklabels([f'{ts:.0f}' for ts in timesteps])    ax.set_yticks(range(0, n_layers, max(1, n_layers//10)))    ax.set_ylabel('Layer Index', fontsize=10)    ax.set_xlabel('Timestep (s)', fontsize=10)    ax.set_title(f'Alpha = {alpha:.2f}', fontsize=12, fontweight='bold')fig2.tight_layout()# Add legend to the second figurefig2.legend(handles=legend_elements, loc='right', bbox_to_anchor=(1.12, 0.5),            ncol=1, fontsize=10, frameon=True)plt.show()

In [ ]:
# Cell: Plot 2x2 Summary (Temperature, dT, Flux, dFlux vs Step Number)# Recreates the plot_results function from convective_flux_v2.py# Input: alpha and timestep# Output: 2x2 subplot showing Temperature, dT, Flux, and dFlux vs step numberimport jsonimport numpy as npimport matplotlib.pyplot as pltimport sysimport os# Add parent directory to path to import convective_flux_v2sys.path.insert(0, os.path.dirname(os.path.abspath('.')))# Import and reload to ensure we have the latest version with history_interval parameterimport importlibif 'convective_grid.convective_flux_v2' in sys.modules:    importlib.reload(sys.modules['convective_grid.convective_flux_v2'])from convective_grid.convective_flux_v2 import run# Parametersalpha = 0.1  # Change this to your desired alphatimestep = 1  # Change this to your desired timestep (in seconds)# Load JSON data to check if run exists (for validation)try:    with open('parameter_sweep_data.json', 'r') as f:        json_data = json.load(f)    # Check if this combination exists in JSON    matching_runs = [run for run in json_data['data']                      if abs(run['alpha'] - alpha) < 1e-6 and abs(run['timestep'] - timestep) < 1e-6]    if len(matching_runs) > 0:        print(f"✓ Found run in JSON data: α={alpha}, dt={timestep}s")        print(f"  Converged: {matching_runs[0]['converged']}, Steps: {matching_runs[0]['steps_to_converge']}")        print("  Note: JSON doesn't contain dT history, so running simulation with save_history=True...")except FileNotFoundError:    print("Note: parameter_sweep_data.json not found. Running simulation directly...")except Exception as e:    print(f"Note: Could not load JSON ({e}). Running simulation directly...")# Guillot profile parameters (same as in data collection)GUILLOT_PARAMS = {    'tint': 150.0,    'tirr': 1200.0,    'kappa_S': 0.01,    'kappa0': 0.02,    'kappa_cia': 0.0,    'beta_S0': 1.0,    'beta_L0': 1.0,    'el1': 3.0/8.0,    'el3': 1.0/3.0}# Run simulation with history tracking enabledprint(f"\nRunning simulation with α={alpha}, dt={timestep}s...")print("=" * 70)# Note: To get data at every step, we need to modify history saving# The run() function saves history at intervals by default (max_steps // 1000)# For now, we'll call run() and see what we get. If we need every step,# we'll need to modify convective_flux_v2.py to use history_interval=1z, T_final, rho_final, P_final, diagnostics = run(    n_layers=100,    max_steps=100000,    alpha=alpha,    dt=timestep,    debug=False,    save_history=True,  # Enable history tracking    history_interval=1,  # Save at every step (not at intervals) to get all 100,000 data points    profile_type="guillot",    guillot_params=GUILLOT_PARAMS,    convergence_tol=1e-10,    check_adiabatic=True,    adiabatic_tolerance=1.0)# Extract history data and create 2x2 plot (same as plot_results function)if all(key in diagnostics for key in ['history_T', 'history_dT', 'history_F', 'history_dF', 'timesteps']):    history_T = diagnostics['history_T']  # Shape: (n_steps, n_interfaces)    history_dT = diagnostics['history_dT']  # Shape: (n_steps, n_interfaces)    history_F = diagnostics['history_F']  # Shape: (n_steps, n_layers)    history_dF = diagnostics['history_dF']  # Shape: (n_steps, n_interfaces)    timesteps = diagnostics['timesteps']    # z is the altitude at interfaces (returned from run())    # z_mid is stored in diagnostics or computed from z    if 'z_mid' in diagnostics:        z_mid = diagnostics['z_mid']    else:        z_mid = (z[:-1] + z[1:]) / 2.0  # Compute layer center altitudes        n_layers = len(z_mid)    n_interfaces = len(z)    n_steps = len(timesteps)        # Debug: Print information about saved steps    print(f"\nHistory data summary:")    print(f"  Total saved snapshots: {n_steps}")    print(f"  Step numbers range: {timesteps[0]} to {timesteps[-1]}")    print(f"  Actual steps taken: {diagnostics.get('steps', 'unknown')}")    print(f"  History interval (estimated): {(timesteps[-1] - timesteps[0]) // (n_steps - 1) if n_steps > 1 else 'N/A'}")        # Determine if we should show legend (hide if too many layers)    show_legend = n_layers <= 10        # Create color map for layers    colors = plt.cm.viridis(np.linspace(0, 1, n_interfaces))    colors_layers = plt.cm.viridis(np.linspace(0, 1, n_layers))        # Create 2x2 subplot figure    fig, axes = plt.subplots(2, 2, figsize=(14, 10))    fig.suptitle(f'Convective Flux Evolution (n_layers={n_layers}, steps={n_steps}, α={alpha}, dt={timestep}s)',                  fontsize=16, fontweight='bold')        # Plot 1: Temperature vs Timestep (at interfaces) - Top Left    ax1 = axes[0, 0]    for i in range(n_interfaces):        label = f'Interface {i} (z={z[i]/1000:.1f} km)' if show_legend else None        ax1.plot(timesteps, history_T[:, i], label=label,                 color=colors[i], linewidth=0.8)    ax1.set_xlabel('Step Number', fontsize=11)    ax1.set_ylabel('Temperature (K)', fontsize=11)    ax1.set_title('Temperature vs Step Number (at Interfaces)', fontsize=12)    if show_legend:        ax1.legend(fontsize=7, loc='best')    ax1.grid(True, alpha=0.3)        # Plot 2: dT vs Timestep (at interfaces) - Top Right    ax2 = axes[0, 1]    for i in range(n_interfaces):        label = f'Interface {i} (z={z[i]/1000:.1f} km)' if show_legend else None        ax2.plot(timesteps, history_dT[:, i], label=label,                 color=colors[i], linewidth=0.8)    ax2.set_xlabel('Step Number', fontsize=11)    ax2.set_ylabel('Temperature Change dT (K)', fontsize=11)    ax2.set_title('Temperature Change vs Step Number (at Interfaces)', fontsize=12)    if show_legend:        ax2.legend(fontsize=7, loc='best')    ax2.grid(True, alpha=0.3)        # Plot 3: Flux vs Timestep (at layer centers) - Bottom Left    ax3 = axes[1, 0]    for i in range(n_layers):        label = f'Layer {i} (z={z_mid[i]/1000:.1f} km)' if show_legend else None        ax3.plot(timesteps, history_F[:, i], label=label,                 color=colors_layers[i], linewidth=0.8)    ax3.set_xlabel('Step Number', fontsize=11)    ax3.set_ylabel('Convective Flux F_conv (erg cm^-2 s^-1)', fontsize=11)    ax3.set_title('Convective Flux vs Step Number (at Layer Centers)', fontsize=12)    if show_legend:        ax3.legend(fontsize=7, loc='best')    ax3.grid(True, alpha=0.3)    ax3.set_yscale('log')  # Log scale for flux        # Plot 4: dFlux (dF/dz) vs Timestep (at interfaces) - Bottom Right    ax4 = axes[1, 1]    for i in range(n_interfaces):        label = f'Interface {i} (z={z[i]/1000:.1f} km)' if show_legend else None        ax4.plot(timesteps, history_dF[:, i], label=label,                 color=colors[i], linewidth=0.8)    ax4.set_xlabel('Step Number', fontsize=11)    ax4.set_ylabel('Flux Divergence dF/dz (erg cm^-3 s^-1)', fontsize=11)    ax4.set_title('Flux Divergence vs Step Number (at Interfaces)', fontsize=12)    #ax4.set_xlim(2000,5000)    #ax4.set_ylim(-100000, 100000)    if show_legend:        ax4.legend(fontsize=7, loc='best')    ax4.grid(True, alpha=0.3)        plt.tight_layout()    plt.show()        # Check for high temperature and show error    max_T = np.max(T_final)    if 'history_T' in diagnostics:        max_T_all_time = np.max([np.max(T_step) for T_step in diagnostics['history_T']])    else:        max_T_all_time = max_T        print("\n" + "=" * 70)    if max_T_all_time > 3000.0:        print("⚠️  WARNING: HIGH TEMPERATURE DETECTED!")        print("=" * 70)        print(f"Maximum temperature during run: {max_T_all_time:.2f} K")        print(f"This exceeds the 3000 K threshold by {max_T_all_time - 3000.0:.2f} K")        print(f"Parameters: α={alpha}, dt={timestep}s")        print("This may indicate numerical instability or unphysical behavior.")    else:        print(f"✓ Maximum temperature: {max_T_all_time:.2f} K (within acceptable range)")        print(f"  Final convergence: {'Converged' if diagnostics.get('converged_adiabatic', False) else 'Not converged'}")        print(f"  Steps taken: {diagnostics.get('steps', len(timesteps))}")        print(f"  Number of layers: {n_layers}")    print("=" * 70 + "\n")else:    print("ERROR: History data not available. Please ensure save_history=True in the run() call.")

In [ ]:
# Cell 5: Adiabaticity Deviation by Layer# Plot |N - N_ad|/N_ad vs altitude for different parameter combinationsimport jsonimport numpy as npimport matplotlib.pyplot as plt# Load datawith open('parameter_sweep_data.json', 'r') as f:    data = json.load(f)# Get unique timestep and alpha valuestimesteps = sorted(set(run['timestep'] for run in data['data']))alphas = sorted(set(run['alpha'] for run in data['data']))# Plot 1: Fixed timestep, varying alphafig1, axes1 = plt.subplots(1, len(timesteps), figsize=(5*len(timesteps), 8), sharey=True)if len(timesteps) == 1:    axes1 = [axes1]# Colors for different alpha valuescolors = plt.cm.viridis(np.linspace(0, 1, len(alphas)))for timestep_idx, timestep in enumerate(timesteps):    ax = axes1[timestep_idx]        # Filter runs for this timestep    runs_at_timestep = [run for run in data['data'] if abs(run['timestep'] - timestep) < 1e-6]        # Plot one line per alpha value    for alpha_idx, alpha in enumerate(alphas):        runs_at_alpha = [run for run in runs_at_timestep if abs(run['alpha'] - alpha) < 1e-6]        if len(runs_at_alpha) == 0:            continue                run = runs_at_alpha[0]                # Get adiabaticity and layer indices        layer_indices = np.arange(len(run['adiabaticity_final']))        adiabaticity = np.array(run['adiabaticity_final'])                ax.plot(adiabaticity,layer_indices, 'o-', color=colors[alpha_idx],                label=f'α={alpha:.2f}', linewidth=2, markersize=3)        ax.set_ylabel('Layer Index', fontsize=12, fontweight='bold')    ax.set_xlabel('|N - N_ad|/N_ad', fontsize=12, fontweight='bold')    ax.set_title(f'Timestep = {timestep:.0f} s', fontsize=13, fontweight='bold')    ax.set_xscale('log')    ax.set_xlim(0,1000)    ax.grid(True, alpha=0.3, which='both')    ax.legend(fontsize=9, loc='best')plt.tight_layout()plt.suptitle('Adiabaticity Deviation by Layer: Varying Alpha (Fixed Timestep)',              fontsize=14, fontweight='bold', y=1.02)plt.show()# Plot 2: Fixed alpha, varying timestepfig2, axes2 = plt.subplots(1, len(alphas), figsize=(5*len(alphas), 8), sharey=True)if len(alphas) == 1:    axes2 = [axes2]# Colors for different timestep valuescolors2 = plt.cm.plasma(np.linspace(0, 1, len(timesteps)))for alpha_idx, alpha in enumerate(alphas):    ax = axes2[alpha_idx]        # Filter runs for this alpha    runs_at_alpha = [run for run in data['data'] if abs(run['alpha'] - alpha) < 1e-6]        # Plot one line per timestep value    for timestep_idx, timestep in enumerate(timesteps):        runs_at_timestep = [run for run in runs_at_alpha if abs(run['timestep'] - timestep) < 1e-6]        if len(runs_at_timestep) == 0:            continue                run = runs_at_timestep[0]                # Get adiabaticity and layer indices        layer_indices = np.arange(len(run['adiabaticity_final']))        adiabaticity = np.array(run['adiabaticity_final'])                ax.plot(adiabaticity, layer_indices,'o-', color=colors2[timestep_idx],                label=f'dt={timestep:.0f}s', linewidth=2, markersize=3)        ax.set_ylabel('Layer Index', fontsize=12, fontweight='bold')    ax.set_xlabel('|N - N_ad|/N_ad', fontsize=12, fontweight='bold')    ax.set_title(f'Alpha = {alpha:.2f}', fontsize=13, fontweight='bold')    ax.set_xscale('log')    ax.grid(True, alpha=0.3, which='both')    ax.legend(fontsize=9, loc='best')    ax.set_xlim(0.0001,1000)plt.tight_layout()plt.suptitle('Adiabaticity Deviation by Layer: Varying Timestep (Fixed Alpha)',              fontsize=14, fontweight='bold', y=1.02)plt.show()

In [ ]:
# Cell 6: Statistical Comparison of Final States# Quantify differences in final states across parameter combinationsimport jsonimport numpy as npimport pandas as pd# Load datawith open('parameter_sweep_data.json', 'r') as f:    data = json.load(f)def calculate_statistical_differences(data):    """    Calculate statistical differences between final states for different parameters.        Compares:    1. Temperature profiles: max|T_final - T_reference| across layers    2. Convective layer overlap: How many convective layers overlap between runs?    3. Adiabaticity difference: max|adiabaticity_final - adiabaticity_reference|    """    results = []        # Use first run as reference    reference_run = data['data'][0]    T_ref_interfaces = np.array(reference_run['T_final'])    T_ref_centers = (T_ref_interfaces[:-1] + T_ref_interfaces[1:]) / 2.0    adiabaticity_ref = np.array(reference_run['adiabaticity_final'])    convective_locs_ref = set(reference_run['convective_locations_final'])        # Compare each run to reference    for run in data['data']:        # Temperature difference        T_run_interfaces = np.array(run['T_final'])        T_run_centers = (T_run_interfaces[:-1] + T_run_interfaces[1:]) / 2.0        max_T_diff = np.max(np.abs(T_run_centers - T_ref_centers))        mean_T_diff = np.mean(np.abs(T_run_centers - T_ref_centers))                # Adiabaticity difference        adiabaticity_run = np.array(run['adiabaticity_final'])        max_adiabaticity_diff = np.max(np.abs(adiabaticity_run - adiabaticity_ref))        mean_adiabaticity_diff = np.mean(np.abs(adiabaticity_run - adiabaticity_ref))                # Convective layer overlap        convective_locs_run = set(run['convective_locations_final'])        overlap = len(convective_locs_ref & convective_locs_run)        overlap_fraction = overlap / len(convective_locs_ref) if len(convective_locs_ref) > 0 else 0.0                results.append({            'alpha': run['alpha'],            'timestep': run['timestep'],            'max_T_diff': max_T_diff,            'mean_T_diff': mean_T_diff,            'max_adiabaticity_diff': max_adiabaticity_diff,            'mean_adiabaticity_diff': mean_adiabaticity_diff,            'convective_overlap': overlap,            'convective_overlap_fraction': overlap_fraction,            'n_convective': run['n_convective_final']        })        return pd.DataFrame(results)# Calculate differencesdf_stats = calculate_statistical_differences(data)# Print summary statisticsprint("=" * 100)print("Statistical Comparison of Final States")print("=" * 100)print("\nReference run: alpha={:.2f}, timestep={:.0f}s".format(    data['data'][0]['alpha'], data['data'][0]['timestep']))print(f"Reference convective layers: {sorted(data['data'][0]['convective_locations_final'])}")print(f"Reference n_convective: {data['data'][0]['n_convective_final']}")print("\n" + "=" * 100)print("Summary Statistics:")print("=" * 100)print(f"\nTemperature Differences (K):")print(f"  Max difference across all runs: {df_stats['max_T_diff'].max():.2f} K")print(f"  Mean difference across all runs: {df_stats['mean_T_diff'].mean():.2f} K")print(f"  Std of mean differences: {df_stats['mean_T_diff'].std():.2f} K")print(f"\nAdiabaticity Differences:")print(f"  Max difference across all runs: {df_stats['max_adiabaticity_diff'].max():.4f}")print(f"  Mean difference across all runs: {df_stats['mean_adiabaticity_diff'].mean():.4f}")print(f"  Std of mean differences: {df_stats['mean_adiabaticity_diff'].std():.4f}")print(f"\nConvective Layer Overlap:")print(f"  Mean overlap fraction: {df_stats['convective_overlap_fraction'].mean():.2%}")print(f"  Runs with 100% overlap: {(df_stats['convective_overlap_fraction'] == 1.0).sum()} / {len(df_stats)}")print(f"  Runs with <50% overlap: {(df_stats['convective_overlap_fraction'] < 0.5).sum()} / {len(df_stats)}")# Print detailed comparison tableprint("\n" + "=" * 100)print("Detailed Comparison (sorted by alpha, then timestep):")print("=" * 100)df_stats_sorted = df_stats.sort_values(['alpha', 'timestep'])print(df_stats_sorted[['alpha', 'timestep', 'max_T_diff', 'mean_T_diff',                        'convective_overlap_fraction', 'n_convective']].to_string(index=False))# Determine if parameters change final state or just convergence timeprint("\n" + "=" * 100)print("CONCLUSION:")print("=" * 100)if df_stats['max_T_diff'].max() < 10.0:  # If temperature differences are small (<10K)    print("✓ TEMPERATURE PROFILES: Parameters appear to affect CONVERGENCE TIME only.")    print(f"  Maximum temperature difference: {df_stats['max_T_diff'].max():.2f} K (very small)")else:    print("✗ TEMPERATURE PROFILES: Parameters affect the FINAL STATE.")    print(f"  Maximum temperature difference: {df_stats['max_T_diff'].max():.2f} K")if df_stats['convective_overlap_fraction'].mean() > 0.8:    print("✓ CONVECTIVE LAYERS: Parameters appear to affect CONVERGENCE TIME only.")    print(f"  Mean overlap: {df_stats['convective_overlap_fraction'].mean():.1%}")else:    print("✗ CONVECTIVE LAYERS: Parameters affect the FINAL STATE.")    print(f"  Mean overlap: {df_stats['convective_overlap_fraction'].mean():.1%}")if df_stats['max_adiabaticity_diff'].max() < 0.1:    print("✓ ADIABATICITY: Parameters appear to affect CONVERGENCE TIME only.")    print(f"  Maximum adiabaticity difference: {df_stats['max_adiabaticity_diff'].max():.4f}")else:    print("✗ ADIABATICITY: Parameters affect the FINAL STATE.")    print(f"  Maximum adiabaticity difference: {df_stats['max_adiabaticity_diff'].max():.4f}")

##### GUILLOT CHECK

In [ ]:
import json# Check if data file exists and what's in itwith open('parameter_sweep_data_guillot.json', 'r') as gf:    gdata = json.load(gf)print("Keys in data:", list(gdata.keys()))if 'data' in gdata:    print(f"Number of runs: {len(gdata['data'])}")    if len(gdata['data']) > 0:        print("First run keys:", list(gdata['data'][0].keys()))        print("Available timesteps:", sorted(set(r['timestep'] for r in gdata['data'])))    else:        print("Data array is empty - need to run data collection!")else:    print("Data structure:", data)

In [ ]:
# Cell 1: Overview Table# Summary table showing key metrics for all parameter combinationsimport jsonimport pandas as pdimport numpy as np# Load datawith open('parameter_sweep_data_guillot.json', 'r') as gf:    gdata = json.load(gf)# Create summary tablegsummary_data = []for run in gdata['data']:    # Calculate mean temperature from T_final (at layer centers)    # T_final is at interfaces, need to get layer centers    T_final_interfaces = np.array(run['T_final'])    T_centers = (T_final_interfaces[:-1] + T_final_interfaces[1:]) / 2.0        gsummary_data.append({        'alpha': run['alpha'],        'timestep': run['timestep'],        'steps_to_converge': run['steps_to_converge'],        'converged': run['converged'],        'n_convective_final': run['n_convective_final'],        'max_grad_diff_final': run['max_grad_diff_final'],        'mean_T_final': np.mean(T_centers),        'min_T_final': np.min(T_centers),        'max_T_final': np.max(T_centers)    })gdf_summary = pd.DataFrame(gsummary_data)# Sort by alpha, then timestep for easy readinggdf_summary = gdf_summary.sort_values(['alpha', 'timestep'])# Display formatted tableprint("=" * 100)print("Parameter Effects Summary Table")print("=" * 100)print(f"\nTotal parameter combinations: {len(gdf_summary)}")print(f"\n{gdf_summary.to_string(index=False)}\n")# Print summary statisticsprint("\n" + "=" * 100)print("Summary Statistics by Parameter")print("=" * 100)# By alphaprint("\nBy Alpha (across all timesteps):")alpha_stats = gdf_summary.groupby('alpha').agg({    'steps_to_converge': ['mean', 'min', 'max'],    'n_convective_final': ['mean', 'min', 'max'],    'max_grad_diff_final': ['mean', 'min', 'max'],    'mean_T_final': ['mean', 'std']}).round(2)print(alpha_stats)# By timestepprint("\nBy Timestep (across all alpha values):")timestep_stats = gdf_summary.groupby('timestep').agg({    'steps_to_converge': ['mean', 'min', 'max'],    'n_convective_final': ['mean', 'min', 'max'],    'max_grad_diff_final': ['mean', 'min', 'max'],    'mean_T_final': ['mean', 'std']}).round(2)print(timestep_stats)

In [ ]:
# Plot 2: Fixed alpha, varying timestep# Load data and set up if neededimport jsonimport numpy as npimport matplotlib.pyplot as pltfrom matplotlib.colors import ListedColormapfrom matplotlib.patches import Rectangle, Patchwith open('parameter_sweep_data_guillot.json', 'r') as gf:    gdata = json.load(gf)timesteps = sorted(set(run['timestep'] for run in gdata['data']))alphas = sorted(set(run['alpha'] for run in gdata['data']))n_layers = gdata['data'][0]['n_layers']def classify_layer_type_custom(N, N_ad):    classifications = np.empty(len(N), dtype=object)    convective_mask = N > 1.5 * N_ad    classifications[convective_mask] = 'convective'    adiabatic_mask = (N >= N_ad) & (N <= 1.5 * N_ad)    classifications[adiabatic_mask] = 'adiabatic'    radiative_mask = N < N_ad    classifications[radiative_mask] = 'radiative'    return classificationscolors_map = ['blue', 'yellow', 'green']cmap = ListedColormap(colors_map)legend_elements = [    Patch(facecolor='green', label='Convective'),    Patch(facecolor='yellow', label='Adiabatic (N_ad ≤ N ≤ 1.5*N_ad)'),    Patch(facecolor='blue', label='Radiative'),    Patch(facecolor='white', edgecolor='red', hatch='///', linewidth=1.5, label='T > 3000K'),    Patch(facecolor='white', edgecolor='lightblue', hatch='///', linewidth=1.5, label='T < 100K')]fig2, axes2 = plt.subplots(1, len(alphas), figsize=(5*len(alphas), 10), sharey=True)if len(alphas) == 1:    axes2 = [axes2]for alpha_idx, alpha in enumerate(alphas):    ax = axes2[alpha_idx]        # Filter runs for this alpha    runs_at_alpha = [run for run in gdata['data'] if abs(run['alpha'] - alpha) < 1e-6]        # Create grid: layers (rows) vs timestep (columns)    layer_type_grid = np.zeros((n_layers, len(timesteps)), dtype=int)    high_T_mask = np.zeros((n_layers, len(timesteps)), dtype=bool)    low_T_mask = np.zeros((n_layers, len(timesteps)), dtype=bool)        for timestep_idx, timestep in enumerate(timesteps):        runs_at_timestep = [run for run in runs_at_alpha if abs(run['timestep'] - timestep) < 1e-6]        if len(runs_at_timestep) == 0:            continue                run = runs_at_timestep[0]                # Recalculate layer types using custom criterion        N_final = np.array(run['N_final'])        N_ad = run['N_ad']        layer_types = classify_layer_type_custom(N_final, N_ad)                # Get temperature at layer centers        T_final_interfaces = np.array(run['T_final'])        T_centers = (T_final_interfaces[:-1] + T_final_interfaces[1:]) / 2.0                # Map layer types to integers        for layer_idx, layer_type in enumerate(layer_types):            if layer_type == 'radiative':                layer_type_grid[layer_idx, timestep_idx] = 0            elif layer_type == 'adiabatic':                layer_type_grid[layer_idx, timestep_idx] = 1            elif layer_type == 'convective':                layer_type_grid[layer_idx, timestep_idx] = 2                        if T_centers[layer_idx] > 3000.0:                high_T_mask[layer_idx, timestep_idx] = True                        if T_centers[layer_idx] < 100.0:                low_T_mask[layer_idx, timestep_idx] = True        # Create heatmap    im = ax.imshow(        layer_type_grid,        aspect='auto',        cmap=cmap,        vmin=0,        vmax=2,        interpolation='nearest',        origin='lower',        extent=[-0.5, len(timesteps)-0.5, -0.5, n_layers-0.5]    )        # Overlay hatching for temperature conditions    cell_width = 1.0    cell_height = 1.0    for layer_idx in range(n_layers):        for timestep_idx in range(len(timesteps)):            if high_T_mask[layer_idx, timestep_idx]:                rect = Rectangle(                    (timestep_idx - 0.5, layer_idx - 0.5),                    cell_width,                    cell_height,                    fill=False,                    hatch='///',                    edgecolor='red',                    linewidth=1.5,                    alpha=0.8                )                ax.add_patch(rect)                        if low_T_mask[layer_idx, timestep_idx]:                rect = Rectangle(                    (timestep_idx - 0.5, layer_idx - 0.5),                    cell_width,                    cell_height,                    fill=False,                    hatch='///',                    edgecolor='lightblue',                    linewidth=1.5,                    alpha=0.8                )                ax.add_patch(rect)        # Add vertical lines between timestep values    for i in range(len(timesteps) - 1):        ax.axvline(x=i + 0.5, color='white', linestyle='--', linewidth=1, alpha=1)        # Set ticks and labels    ax.set_xticks(range(len(timesteps)))    ax.set_xticklabels([f'{ts:.0f}' for ts in timesteps])    ax.set_yticks(range(0, n_layers, max(1, n_layers//10)))    ax.set_ylabel('Layer Index', fontsize=10)    ax.set_xlabel('Timestep (s)', fontsize=10)    ax.set_title(f'Alpha = {alpha:.2f}', fontsize=12, fontweight='bold')fig2.tight_layout()# Add legend to the second figurefig2.legend(    handles=legend_elements,    loc='right',    bbox_to_anchor=(1.12, 0.5),    ncol=1,    fontsize=10,    frameon=True)plt.show()

In [ ]:
# Cell 6: Statistical Comparison of Final States# Quantify differences in final states across parameter combinationsimport jsonimport numpy as npimport pandas as pd# Load datawith open('parameter_sweep_data_guillot.json', 'r') as gf:    gdata = json.load(gf)def calculate_statistical_differences(gdata):    """    Calculate statistical differences between final states for different parameters.        Compares:    1. Temperature profiles: max|T_final - T_reference| across layers    2. Convective layer overlap: How many convective layers overlap between runs?    3. Adiabaticity difference: max|adiabaticity_final - adiabaticity_reference|    """    results = []        # Use first run as reference    reference_run = gdata['data'][0]    T_ref_interfaces = np.array(reference_run['T_final'])    T_ref_centers = (T_ref_interfaces[:-1] + T_ref_interfaces[1:]) / 2.0    adiabaticity_ref = np.array(reference_run['adiabaticity_final'])    convective_locs_ref = set(reference_run['convective_locations_final'])        # Compare each run to reference    for run in gdata['data']:        # Temperature difference        T_run_interfaces = np.array(run['T_final'])        T_run_centers = (T_run_interfaces[:-1] + T_run_interfaces[1:]) / 2.0        max_T_diff = np.max(np.abs(T_run_centers - T_ref_centers))        mean_T_diff = np.mean(np.abs(T_run_centers - T_ref_centers))                # Adiabaticity difference        adiabaticity_run = np.array(run['adiabaticity_final'])        max_adiabaticity_diff = np.max(np.abs(adiabaticity_run - adiabaticity_ref))        mean_adiabaticity_diff = np.mean(np.abs(adiabaticity_run - adiabaticity_ref))                # Convective layer overlap        convective_locs_run = set(run['convective_locations_final'])        overlap = len(convective_locs_ref & convective_locs_run)        overlap_fraction = overlap / len(convective_locs_ref) if len(convective_locs_ref) > 0 else 0.0                results.append({            'alpha': run['alpha'],            'timestep': run['timestep'],            'max_T_diff': max_T_diff,            'mean_T_diff': mean_T_diff,            'max_adiabaticity_diff': max_adiabaticity_diff,            'mean_adiabaticity_diff': mean_adiabaticity_diff,            'convective_overlap': overlap,            'convective_overlap_fraction': overlap_fraction,            'n_convective': run['n_convective_final']        })        return pd.DataFrame(results)# Calculate differencesdf_stats = calculate_statistical_differences(gdata)# Print summary statisticsprint("=" * 100)print("Statistical Comparison of Final States")print("=" * 100)print("\nReference run: alpha={:.2f}, timestep={:.3f}s".format(    gdata['data'][0]['alpha'], gdata['data'][0]['timestep']))print(f"Reference convective layers: {sorted(gdata['data'][0]['convective_locations_final'])}")print(f"Reference n_convective: {gdata['data'][0]['n_convective_final']}")print("\n" + "=" * 100)print("Summary Statistics:")print("=" * 100)print(f"\nTemperature Differences (K):")print(f"  Max difference across all runs: {df_stats['max_T_diff'].max():.2f} K")print(f"  Mean difference across all runs: {df_stats['mean_T_diff'].mean():.2f} K")print(f"  Std of mean differences: {df_stats['mean_T_diff'].std():.2f} K")print(f"\nAdiabaticity Differences:")print(f"  Max difference across all runs: {df_stats['max_adiabaticity_diff'].max():.4f}")print(f"  Mean difference across all runs: {df_stats['mean_adiabaticity_diff'].mean():.4f}")print(f"  Std of mean differences: {df_stats['mean_adiabaticity_diff'].std():.4f}")print(f"\nConvective Layer Overlap:")print(f"  Mean overlap fraction: {df_stats['convective_overlap_fraction'].mean():.2%}")print(f"  Runs with 100% overlap: {(df_stats['convective_overlap_fraction'] == 1.0).sum()} / {len(df_stats)}")print(f"  Runs with <50% overlap: {(df_stats['convective_overlap_fraction'] < 0.5).sum()} / {len(df_stats)}")# Print detailed comparison tableprint("\n" + "=" * 100)print("Detailed Comparison (sorted by alpha, then timestep):")print("=" * 100)df_stats_sorted = df_stats.sort_values(['alpha', 'timestep'])print(df_stats_sorted[['alpha', 'timestep', 'max_T_diff', 'mean_T_diff',                        'convective_overlap_fraction', 'n_convective']].to_string(index=False))# Determine if parameters change final state or just convergence timeprint("\n" + "=" * 100)print("CONCLUSION:")print("=" * 100)if df_stats['max_T_diff'].max() < 10.0:    print("✓ TEMPERATURE PROFILES: Parameters appear to affect CONVERGENCE TIME only.")    print(f"  Maximum temperature difference: {df_stats['max_T_diff'].max():.2f} K")else:    print("✗ TEMPERATURE PROFILES: Parameters affect the FINAL STATE.")    print(f"  Maximum temperature difference: {df_stats['max_T_diff'].max():.2f} K")if df_stats['convective_overlap_fraction'].mean() > 0.8:    print("✓ CONVECTIVE LAYERS: Parameters appear to affect CONVERGENCE TIME only.")    print(f"  Mean overlap: {df_stats['convective_overlap_fraction'].mean():.1%}")else:    print("✗ CONVECTIVE LAYERS: Parameters affect the FINAL STATE.")    print(f"  Mean overlap: {df_stats['convective_overlap_fraction'].mean():.1%}")if df_stats['max_adiabaticity_diff'].max() < 0.1:    print("✓ ADIABATICITY: Parameters appear to affect CONVERGENCE TIME only.")    print(f"  Maximum adiabaticity difference: {df_stats['max_adiabaticity_diff'].max():.4f}")else:    print("✗ ADIABATICITY: Parameters affect the FINAL STATE.")    print(f"  Maximum adiabaticity difference: {df_stats['max_adiabaticity_diff'].max():.4f}")

# Code Block 1: Timescale Evolution Plot for Selected LayerPlot convective and radiative timescales vs timestep for a selected layer.

# Convective vs Radiative Timescales

In [ ]:
import jsonimport numpy as npimport matplotlib.pyplot as plt# Parameters to plotALPHA_VALUES_PLOT = [0.01, 0.1, 1.0]TIMESTEP_VALUES_PLOT = [0.1, 1, 10]# Load datawith open('parameter_sweep_data_guillot.json', 'r') as f:    data = json.load(f)# Prepare figurefig, axes = plt.subplots(3, 3, figsize=(15, 12), sharey=True)fig.subplots_adjust(hspace=0.3, wspace=0.2)for i, alpha in enumerate(ALPHA_VALUES_PLOT):    for j, dt in enumerate(TIMESTEP_VALUES_PLOT):        ax = axes[i, j]                # Filter runs for this alpha and dt        filtered_runs = [run for run in data['data']                          if abs(run['alpha'] - alpha) < 1e-6 and abs(run['timestep'] - dt) < 1e-6]                if len(filtered_runs) == 0:            ax.set_title(f'α={alpha}, dt={dt}\n(No data)')            ax.set_xlabel('Layer')            if j == 0:                ax.set_ylabel('Timescale (s)')            continue                run = filtered_runs[0]  # Assume only one run per combination                n_layers = run['n_layers']        layers = np.arange(n_layers)                # Convective timescale        t_conv = np.array([v if v is not None else np.nan for v in run.get('t_conv_final', [None]*n_layers)])                # -----------------------------        # Report null convective layers        # -----------------------------        null_conv_mask = ~np.isfinite(t_conv)        if np.any(null_conv_mask):            z_int = run.get('z_interfaces_km')            P_int = run.get('P_final')            # Guard against missing / null interface data            if z_int is None or P_int is None:                print(f"\nα={alpha}, dt={dt} | NULL convective layers, but z/P data missing")                continue            z_interfaces_km = np.asarray(z_int)            P_interfaces = np.asarray(P_int)            if z_interfaces_km.ndim != 1 or P_interfaces.ndim != 1:                print(f"\nα={alpha}, dt={dt} | NULL convective layers, but z/P malformed")                continue            # Layer centers            z_centers_km = 0.5 * (z_interfaces_km[:-1] + z_interfaces_km[1:])            P_centers = 0.5 * (P_interfaces[:-1] + P_interfaces[1:])            print(f"\nα={alpha}, dt={dt} | NULL convective timescale layers:")            for idx in np.where(null_conv_mask)[0]:                print(                    f"  Layer {idx + 1:3d} | "                    f"z = {z_centers_km[idx]:8.3f} km | "                    f"P = {P_centers[idx]:.3e} Pa"                )        # Radiative timescale        tau_rad = np.array([v if v is not None else np.nan for v in run.get('tau_rad_final', [None]*n_layers)])                # Plot convective timescale - red dots        finite_conv = np.isfinite(t_conv)        ax.scatter(layers[finite_conv], t_conv[finite_conv], color='red', s=20, label='Convective', zorder=10)                # Plot radiative timescale - blue crosses        finite_rad = np.isfinite(tau_rad)        ax.scatter(layers[finite_rad], tau_rad[finite_rad], color='blue', marker='x', s=40, label='Radiative', zorder=10)                # Add max_grad_diff_final as text        max_grad_diff = run.get('max_grad_diff_final', None)        if max_grad_diff is not None:            ax.text(0.02, 0.5, f"max_grad_diff_final = {max_grad_diff:.7g}",                     transform=ax.transAxes, fontsize=9, verticalalignment='center', color='black')                ax.set_yscale('log')        ax.set_xlabel('Layer')        if j == 0:            ax.set_ylabel('Timescale (s)')        ax.set_title(f'α={alpha}, dt={dt}')        ax.grid(True, linestyle='--', alpha=0.3)        ax.legend(fontsize=8)plt.tight_layout()plt.show()

### Adiabat Overplot

In [ ]:
import jsonimport numpy as npimport matplotlib.pyplot as plt# -----------------------------# Parameters to plot# -----------------------------ALPHA_VALUES = [0.01, 0.1, 0.5]DT_VALUES = [0.01, 0.1, 1.0]# -----------------------------# Load data# -----------------------------with open('parameter_sweep_data_guillot.json', 'r') as f:    data = json.load(f)# -----------------------------# Set up figure# -----------------------------fig, axes = plt.subplots(    nrows=len(ALPHA_VALUES),    ncols=len(DT_VALUES),    figsize=(15, 18),    sharey=True)# -----------------------------# Loop over alpha / dt# -----------------------------for i, alpha in enumerate(ALPHA_VALUES):    for j, dt in enumerate(DT_VALUES):        ax = axes[i, j]        runs = [            run for run in data['data']            if run.get('T_final') is not None            and run.get('z_interfaces_km') is not None            and abs(run.get('alpha', -1) - alpha) < 1e-6            and abs(run.get('timestep', -1) - dt) < 1e-6        ]        if len(runs) == 0:            ax.set_title(f'α={alpha}, dt={dt}\n(No data)')            continue        run = runs[0]        # -----------------------------        # Layer-center quantities        # -----------------------------        T_interfaces = np.array(run['T_final'])        z_interfaces_km = np.array(run['z_interfaces_km'])        z_interfaces_m = z_interfaces_km * 1000.0        T_centers = 0.5 * (T_interfaces[:-1] + T_interfaces[1:])        z_centers = 0.5 * (z_interfaces_m[:-1] + z_interfaces_m[1:])        # -----------------------------        # Adiabat using stored N_ad        # -----------------------------        if 'N_ad' not in run:            raise RuntimeError("N_ad not found in run data")        Gamma = run['N_ad']  # K/m        base_idx = np.argmin(z_centers)        T0 = T_centers[base_idx]        z0 = z_centers[base_idx]        T_adiabat = T0 - Gamma * (z_centers - z0)                # -----------------------------        # Deviation from adiabat (>5%)        # -----------------------------        frac_dev = np.abs(T_centers - T_adiabat) / T_adiabat        bad_mask = frac_dev > 0.01        # -----------------------------        # Plot        # -----------------------------        ax.plot(            T_centers,            z_centers / 1000.0,            'o-',            linewidth=2,            markersize=4,            label='Model'        )        ax.plot(            T_adiabat,            z_centers / 1000.0,            '--',            color='black',            linewidth=2,            label='Dry adiabat'        )        ax.set_title(f'α={alpha}, dt={dt}')        ax.set_xlim(600, 1200)        ax.grid(True, alpha=0.3)        if i == len(ALPHA_VALUES) - 1:            ax.set_xlabel('Temperature (K)', fontsize=11, fontweight='bold')        if j == 0:            ax.set_ylabel('Altitude (km)', fontsize=11, fontweight='bold')        if i == 0 and j == 0:            ax.legend(fontsize=9)        if 'P_final' in run:            P_interfaces = np.array(run['P_final'])            P_centers = 0.5 * (P_interfaces[:-1] + P_interfaces[1:])            # Plot deviation points (temperature vs altitude)            ax.scatter(                T_centers[bad_mask],                z_centers[bad_mask] / 1000.0,                color='red',                s=30,                zorder=5,                label='>5% off adiabat' if (i == 0 and j == 0) else None            )            # Optional: print pressure + altitude for debugging            print(                f"α={alpha}, dt={dt} | Deviations (>1%):"            )            for zz, pp in zip(z_centers[bad_mask], P_centers[bad_mask]):                print(f"  z = {zz/1000:.2f} km, P = {pp:.3e} Pa")# -----------------------------# Final layout# -----------------------------plt.tight_layout()plt.suptitle(    'Temperature Profiles with Adiabat',    fontsize=16,    fontweight='bold',    y=1.02)plt.show()